In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
import json
import numpy as np
data_dir = '/project/danielsf'
valid_visit_path = os.path.join(data_dir, 'valid_hsc_visits.txt')
assert os.path.isfile(valid_visit_path)
valid_data_id_list = []
with open(valid_visit_path, 'r') as in_file:
    for line in in_file:
        data_id = json.loads(line)
        valid_data_id_list.append(data_id)

In [2]:
unq_id = set()
for data_id in valid_data_id_list:
    visit_ccd = (data_id['visit'], data_id['ccd'])
    assert visit_ccd not in unq_id
    unq_id.add(visit_ccd)

In [3]:
len(unq_id)

473680

In [4]:
len(valid_data_id_list)

473680

In [5]:
from lsst.daf.persistence import Butler
repo_dir = '/datasets/hsc/repo/rerun/DM-13666/WIDE'
butler = Butler(repo_dir)

In [6]:
import time
from astropy.coordinates import SkyCoord
from astropy import units as u

In [19]:
print(valid_data_id_list[0])

{'ccd': 0, 'filter': 'HSC-Y', 'visit': 374}


In [23]:
t_start = time.time()
bounds = {}
ct = 0
#theta = np.arange(0.0,2.0*np.pi,0.01)

n_data = len(valid_data_id_list)
ccd = np.zeros(n_data, dtype=int)
visit = np.zeros(n_data, dtype=int)
hsc_filter = np.empty(n_data, dtype=(str,5))
ra_center = np.zeros(n_data, dtype=float)
dec_center = np.zeros(n_data, dtype=float)
ra_min = np.zeros(n_data, dtype=float)
ra_max = np.zeros(n_data, dtype=float)
dec_min = np.zeros(n_data, dtype=float)
dec_max = np.zeros(n_data, dtype=float)
for ii, data_id in enumerate(valid_data_id_list):
    tag = (data_id['visit'], data_id['ccd'])
    src = butler.get('src', dataId=data_id)
    _ra_max = src['coord_ra'].max()
    _ra_min = src['coord_ra'].min()
    _dec_max = src['coord_dec'].max()
    _dec_min = src['coord_dec'].min()
    _ra_center = 0.5*(_ra_max+_ra_min)
    _dec_center = 0.5*(_dec_max+_dec_min)

    ccd[ii] = data_id['ccd']
    visit[ii] = data_id['visit']
    hsc_filter[ii] = data_id['filter']
    ra_center[ii] = _ra_center
    dec_center[ii] = _dec_center
    ra_min[ii] = _ra_min
    ra_max[ii] = _ra_max
    dec_min[ii] = _dec_min
    dec_max[ii] = _dec_max
    
    #center_pt = SkyCoord(ra_center*u.rad,
    #                     dec_center*u.rad)
    #astropy_src = SkyCoord(src['coord_ra']*u.rad,
    #                       src['coord_dec']*u.rad)
    #dist = center_pt.separation(astropy_src)
    #max_dist = dist.max()/u.deg
    #plt.figure()
    #plt.scatter(np.degrees(src['coord_ra']), np.degrees(src['coord_dec']), color='b')
    #plt.scatter(np.degrees(ra_center), np.degrees(dec_center), color='r')
    #plt.scatter(np.degrees(ra_center)+max_dist*np.cos(theta),
    #            np.degrees(dec_center)+max_dist*np.sin(theta), color='r')
    #plt.show()

    #bounds[tag] = (ra_center, dec_center, ra_min, ra_max, dec_min, dec_max)
    
    ct += 1
    if ct%100 == 0:
        duration = (time.time()-t_start)/3600.0
        per = duration/ct
        pred = per*len(valid_data_id_list)
        print(ct,duration,per,pred)
    if ct>500:
        break
    

100 0.0007405369149314033 7.405369149314033e-06 3.5077752586470714
200 0.001877575914065043 9.387879570325215e-06 4.446850794871648
300 0.002601358493169149 8.671194977230496e-06 4.107371636814541
400 0.0036619530783759224 9.154882695939807e-06 4.336484835412768
500 0.004429953826798333 8.859907653596667e-06 4.196761057355669


In [41]:
out_dir = '/project/danielsf'
assert os.path.isdir(out_dir)

In [42]:
out_name = os.path.join(out_dir, 'valid_hsc_visit_extent.h5')
assert not os.path.exists(out_name)

In [45]:
with h5py.File(out_name, 'w') as out_file:
    out_file.create_dataset('visit', data = visit)
    out_file.create_dataset('ccd', data=ccd)
    out_file.create_dataset('filter', data=hsc_filter.astype(bytes))
    out_file.create_dataset('ra_center', data=ra_center)
    out_file.create_dataset('dec_center', data=dec_center)
    out_file.create_dataset('ra_min', data=ra_min)
    out_file.create_dataset('ra_max', data=ra_max)
    out_file.create_dataset('dec_min', data=dec_min)
    out_file.create_dataset('dec_max', data=dec_max)

103

In [46]:
with h5py.File(out_name, 'r') as in_file:
    print(in_file['filter'][10])

b'HSC-Y'


2.4558171800393103